In [ ]:
# Importação das bibliotecas necessárias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import joblib

# Carregar o dataset
# Certifique-se de que o arquivo 'diabetes.csv' está na mesma pasta que o seu notebook
data = r"C:\Users\Nobre\Documents\Ciencia da Computacao - UNIPE\Fábrica\Git\WorkshopDados-25.2\atividade_machine_learning_1009\diabetes.csv"
df = pd.read_csv(data)

# --- 1. Análise e Exploração dos Dados ---
print("Formato do DataFrame:", df.shape)
print("\nPrimeiras 5 linhas:")
print(df.head())
print("\nInformações Gerais:")
df.info()
print("\nEstatísticas Descritivas:")
print(df.describe())

# Verificando o balanceamento da variável alvo
print("\nDistribuição da variável 'Outcome':")
print(df['Outcome'].value_counts())

# Visualização da distribuição da variável 'Outcome'
sns.countplot(x='Outcome', data=df)
plt.title('Distribuição da Variável Outcome (0 = Não Diabético, 1 = Diabético)')
plt.show()

# Matriz de correlação para entender a relação entre as variáveis
plt.figure(figsize=(12, 10))
sns.heatmap(df.corr(), annot=True, cmap='Wistia')
plt.title('Matriz de Correlação')
plt.show()


# --- 2. Tratamento dos Dados ---
# Valores '0' em certas colunas podem ser considerados dados ausentes.
# Vamos substituí-los pela média da respectiva coluna para não perder dados.
zero_fields = ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']
for col in zero_fields:
    mean_val = df[col].mean(skipna=True)
    df[col] = df[col].replace(0, mean_val)

# --- 3. Separação dos Dados em Features (X) e Target (y) ---
X = df.drop('Outcome', axis=1)
y = df['Outcome']

# --- 4. Pré-processamento: Padronização das Features ---
# É importante para algoritmos como Regressão Logística e SVM
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# --- 5. Divisão da Base entre Treino e Teste ---
# Usamos stratify=y para manter a proporção das classes no treino e teste
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42, stratify=y)

print(f"\nFormato dos dados de treino: {X_train.shape}")
print(f"Formato dos dados de teste: {X_test.shape}")

# --- 6. Definição, Treinamento e Validação de Vários Modelos ---
models = {
    "Regressão Logística": LogisticRegression(max_iter=1000),
    "K-Nearest Neighbors (KNN)": KNeighborsClassifier(),
    "Support Vector Machine (SVM)": SVC(),
    "Random Forest": RandomForestClassifier(random_state=42)
}

for name, model in models.items():
    # Treinamento
    print(f"\n--- Treinando o modelo: {name} ---")
    model.fit(X_train, y_train)
    
    # Previsão e Validação
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)
    
    print(f"Acurácia: {accuracy:.4f}")
    print("Relatório de Classificação:")
    print(classification_report(y_test, y_pred))
    
    # Matriz de Confusão
    cm = confusion_matrix(y_test, y_pred)
    plt.figure(figsize=(6, 4))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
    plt.title(f'Matriz de Confusão - {name}')
    plt.xlabel('Previsto')
    plt.ylabel('Verdadeiro')
    plt.show()

# --- 7. Salvando a Solução ---
# Com base nos resultados, o Random Forest geralmente apresenta um bom equilíbrio.
# Vamos salvar este modelo como exemplo.
best_model = models["Random Forest"]
joblib.dump(best_model, 'random_forest_diabetes_model.pkl')

print("\nModelo Random Forest salvo com sucesso como 'random_forest_diabetes_model.pkl'")